<a href="https://colab.research.google.com/github/ZoneSixGames/ML-Prompt-Scripts/blob/main/Halloween_Madlib_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🧠 Madlib Prompts
#@markdown When enabled, prompts will be generated at random using a templating language. This allows for very expressive prompts and combines well with batch runs but does require doing some code edits.
#@markdown - Check out the [demo notebook](https://colab.research.google.com/drive/1gGwD0zDvyx0OnJU4KLjE8HrwnTlzI2rS?usp=sharing) for more explanation and examples
AllowSimpleMadlibSyntax = True #on by default, to support {|} format in prompts
UseFullMadlibTemplates = True #@param {type:'boolean'}

## Template definition setup

ml_templates = [
  { 
      "prompt" : "A {adj2} {color} {character} {verb} in a {adj} {scene} by {Salvador Dali| Francisco de Goya | Leonardo DaVinci | Francis Bacon | Quinten Massys | Beeple | Hieronymus Bosch}", 
      "lookup" : {
          "color" :['translucent', 'white'],
          "character" :['skeleton', 'ghost', 'marsmellow man']
      }, 
      "weight": 1 # increase the frequency of this prompt
  },
  {   
      "prompt" : "A group of {adj2} {character}s {verb} a {victim} in a {adj} {color} {scene} by {Salvador Dali| Francisco de Goya | Leonardo DaVinci | Francis Bacon | Quinten Massys | Beeple | Hieronymus Bosch}", 
      "lookup" : {},
      "weight" : 2
  },
  {   
      "prompt" : "A {adj2} {character} {verb} {victim} in a {adj} {scene} by {Salvador Dali| Francisco de Goya | Leonardo DaVinci | Francis Bacon | Quinten Massys | Beeple | Hieronymus Bosch}",
      "lookup" : {},
      "weight" : 2
  }
]
ml_lookup = {
    "adj":['dark','enchanted','forbidden','rotten', 'horrific', 'dirty', 'bloody', 'haunted'],
    "adj2" : ['dead','scary', 'snarling','smiling','crazy', 'bloody', 'ghastly', 'murderous', 'wicked', 'evil','futuristic'],
    "character":['skeleton','monster','ghost','zombie', 'witch', 'wizard', 'vampire', 'bat', 'werewolf', 'black cat', 'leprechaun' 'Jack-o-Lantern', 'marshmellow man', 'death robot cyborg', 'clown', 'pirate'],
    "scene":['cemetary', 'pirate ship', 'castle', 'foggy forest', 'swamp', 'old house', '1970s vintage style trailer', 'dungeon', 'satanic circle', 'long dark candle lit hallway', 'a dank basement with cobwebs', 'a dusty attic'],
    "color" : ['black','red','grey','green', 'orange'], 
    "verb" : ['feasting on', 'stabbing', 'gathering', 'slaughtering', 'waving at', 'jumping on', 'attacking', 'hexing', 'strangling', 'tying up'],
    "victim" : ['a hacker', 'an old man', 'a scientist', 'a Pepe frog', 'an old lady', 'a sexy woman', 'a high school jock', 'a cannabis plant', 'a human', 'a nerd', 'a businessman', 'a king', 'a unicorn']
}
ml_options = {
    "avoid_repeats" : True, 
    "fix_indefinite_articles" : True 
} 

## -------------------------------- 
## supporting code from linked notebook

import re
import random
import copy

def madlib_prompt(base, lookup = {},
        options = { 
            "avoid_repeats" : True, 
            "fix_indefinite_articles" : True 
        }):
    lookup_cardstack = {}

    def on_madlib_sub( match ):
        opt = match.group(2).split('|')

        key = opt[0]

        # do a lookup if there's only one option in the brackets
        if len(opt) == 1 and lookup.get(key):

            # discard used words to avoid repeats, unless no words remain
            if options["avoid_repeats"]:

                if len(lookup_cardstack.get(key) or []) == 0:
                    lookup_cardstack[key] = copy.copy( lookup[key] )

                g2 = random.choice( lookup_cardstack[key] )
                lookup_cardstack[key].remove(g2)

               
            else:
                g2 = random.choice( lookup.get(key) )

        # choose one of the options to fill this space
        else:
            g2 = random.choice( opt )

        # if the previous word is 'A' or 'An', figure out if the 'n' is needed
        g1 = match.group(1)
        if g1 is not None:
            if options["fix_indefinite_articles"]:
                if g2[0].lower() in list("aeiou"):
                    g1 = g1[0] + 'n'
                else:
                    g1 = g1[0]
            g2 = g1 + ' ' + g2
        
        return g2
    
    # find madlib spots, and replace
    return re.sub(r"(\b[Aa]n? )?{([^{}]*)}", on_madlib_sub, base )

def madlib_template( templates, global_lookup = {}, 
        options = { 
            "avoid_repeats" : True, 
            "fix_indefinite_articles" : True 
        }):

    # get prompt template weights and pick one
    weights = [ p.get('weight') or 1 for p in templates]
    t = random.choices( templates, weights=weights, k=1 )[0]
    
    lookup = { **global_lookup, **(t.get('lookup') or {}) }
    lookup_cardstack = {}

    def on_madlib_sub( match ):
        opt = match.group(2).split('|')

        key = opt[0]

        # do a lookup if there's only one option in the brackets
        if len(opt) == 1 and lookup.get(key):

            # discard used words to avoid repeats, unless no words remain
            if options["avoid_repeats"]:

                if len(lookup_cardstack.get(key) or []) == 0:
                    lookup_cardstack[key] = copy.copy( lookup[key] )

                g2 = random.choice( lookup_cardstack[key] )
                lookup_cardstack[key].remove(g2)

            else:
                g2 = random.choice( lookup.get(key) )
          
        # or just pick one of the given options  
        else:
            g2 = random.choice( opt )

        g1 = match.group(1)
        if g1 is not None:

            # if the previous word is 'A' or 'An', figure out if the 'n' is needed
            if options["fix_indefinite_articles"]:
                    if g2[0].lower() in list("aeiou"):
                        g1 = g1[0] + 'n'
                    else:
                        g1 = g1[0]
            g2 = g1 + ' ' + g2
            
        return g2
    
    # find madlib spots, and replace
    return re.sub(r"(\b[Aa]n? )?{([^{}]*)}", on_madlib_sub, t['prompt'] )

## -------------------------------- 
## hook into the generation
def get_prompt():

    if UseFullMadlibTemplates:
        # generate prompt from template        
        if 'base_prompt' in run_args:
            del run_args.base_prompt
        return madlib_template( ml_templates, ml_lookup, ml_options )
        
    # should still use simple madlibbing functionality 
    # when not using an advanced template
    if AllowSimpleMadlibSyntax:
        return madlib_prompt(run_args.base_prompt)

    return run_args.base_prompt

